## Extraer datos mediante la API de inmobiliaria

pagina donde se hizo el webscraping: https://www.argenprop.com/

## Paises admitidos:
#### Argentina, Chile, Uruguay, Brasil

ojo: brasil solo tiene 1 propiedad

## 1. Creamos la conexión con la API para extraer los datos

In [1]:
%%time
import requests
import pandas as pd

url = 'http://127.0.0.1:5000/argenprop'

# Parámetros de la solicitud
params = {'pais': 'argentina', 'limite': 300}
response = requests.get(url, params=params)

data = response.json()
df = pd.DataFrame(data)

CPU times: total: 406 ms
Wall time: 1min 13s
